In [1]:
%run ../__init__.py

## Data preprocessing with Tfidf

In [2]:
budget_train = pd.read_csv('../data/train_cleaned.csv', index_col=0)
budget_test = pd.read_csv('../data/test_clean.csv', index_col=0)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (6,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
for col in budget_test[word_predictors]:
    budget_test[col] = budget_test[word_predictors].loc[:,[col]].fillna(' ')

for col in budget_train[word_predictors]:
    budget_train[col] = budget_train[word_predictors].loc[:,[col]].fillna(' ')

In [4]:
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    Parameters
     ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]
    

# feature union transformer list 
transformer_list = [
    ('Object_Description', Pipeline([
                ('selector', ItemSelector(key='Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_2', Pipeline([
                ('selector', ItemSelector(key='Text_2')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('SubFund_Description', Pipeline([
                ('selector', ItemSelector(key='SubFund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Job_Title_Description', Pipeline([
                ('selector', ItemSelector(key='Job_Title_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_3', Pipeline([
                ('selector', ItemSelector(key='Text_3')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_4', Pipeline([
                ('selector', ItemSelector(key='Text_4')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Sub_Object_Description', Pipeline([
                ('selector', ItemSelector(key='Sub_Object_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Location_Description', Pipeline([
                ('selector', ItemSelector(key='Location_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Function_Description', Pipeline([
                ('selector', ItemSelector(key='Function_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Facility_or_Department', Pipeline([
                ('selector', ItemSelector(key='Facility_or_Department')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Position_Extra', Pipeline([
                ('selector', ItemSelector(key='Position_Extra')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Program_Description', Pipeline([
                ('selector', ItemSelector(key='Program_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Fund_Description', Pipeline([
                ('selector', ItemSelector(key='Fund_Description')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
    ('Text_1', Pipeline([
                ('selector', ItemSelector(key='Text_1')),
                ('tfidf', TfidfVectorizer(ngram_range=(1,2), token_pattern='(?u)\\b\\w\\w+\\b',min_df=10)),
            ])),
]

In [5]:
label = LabelEncoder()
X_test = budget_test

#Example of blended model for Operating Status
X_train = budget_train[word_predictors] 
y_train = label.fit_transform(budget_train['Position_Type'])
X_sub_1, X_sub_2, y_sub_1, y_sub_2 = train_test_split(X_train,y_train, test_size = .5, random_state = 42)
X_test = budget_test

lr_pipe_1 = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression())])

lr_pipe_2 = Pipeline([
    ("union", FeatureUnion(transformer_list=transformer_list)),
    ("clf", LogisticRegression(C=10))])

In [6]:
#fit training data   
lr_pipe_1.fit(X_sub_1,y_sub_1)
lr_pipe_2.fit(X_sub_1,y_sub_1)
   

Pipeline(steps=[('union', FeatureUnion(n_jobs=1,
       transformer_list=[('Object_Description', Pipeline(steps=[('selector', ItemSelector(key='Object_Description')), ('tfidf', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='...ty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [7]:
lr1_train_predict = lr_pipe_1.predict_proba(X_sub_2)
lr2_train_predict = lr_pipe_2.predict_proba(X_sub_2)
blended = np.concatenate([lr1_train_predict, lr2_train_predict], axis=1)

# blended.reshape(-1,1)

# get predict_proba of 2 models for test set and combine them
lr1_test_predict = lr_pipe_1.predict_proba(X_test)
lr2_test_predict = lr_pipe_2.predict_proba(X_test)
test_combine_pred = np.concatenate([lr1_test_predict, lr2_test_predict], axis=1)

In [8]:
# refit blended predict_proba of training set to blender model
blender = RandomForestClassifier()
blender.fit(blended, y_sub_2)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [9]:
train_prediction = blender.predict_proba(blended)
Position_Type_result = [{"target": "Position_Type",
             "log_loss": log_loss(y_sub_2, train_prediction)
             }]
Position_Type_result = pd.DataFrame(Position_Type_result)

# Position_Type_result.to_csv('Position_Type_blended.csv')

#find predict proba with blended model of blended test_proba    
blended_proba = blender.predict_proba(test_combine_pred)

Position_Type = pd.DataFrame(blended_proba,columns=label.classes_)

In [10]:
Position_Type_result

,log_loss,target
0,0.025715,Position_Type


In [11]:
Position_Type

,Position_Type__(Exec) Director,Position_Type__Area Officers,Position_Type__Club Advisor/Coach,Position_Type__Coordinator/Manager,Position_Type__Custodian,Position_Type__Guidance Counselor,Position_Type__Instructional Coach,Position_Type__Librarian,Position_Type__NO_LABEL,Position_Type__Non-Position,...,Position_Type__Principal,Position_Type__Psychologist,Position_Type__School Monitor/Security,Position_Type__Sec/Clerk/Other Admin,Position_Type__Social Worker,Position_Type__Speech Therapist,Position_Type__Substitute,Position_Type__TA,Position_Type__Teacher,Position_Type__Vice Principal
0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
1,0.0,0.0,0.0,0.28,0.0,0.0,0.0,0.0,0.260000,0.060000,...,0.0,0.0,0.0,0.1,0.0,0.0,0.00000,0.000000,0.100000,0.1
2,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.700000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.200000,0.100000,0.0
3,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.700000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.200000,0.100000,0.0
4,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.8,0.0,0.0,0.00000,0.100000,0.000000,0.0
5,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
6,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
7,0.1,0.0,0.0,0.00,0.0,0.0,0.1,0.0,0.016667,0.000000,...,0.0,0.0,0.0,0.1,0.0,0.0,0.00000,0.000000,0.000000,0.0
8,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,1.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.000000,0.000000,0.0
9,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.650000,0.040000,...,0.0,0.0,0.0,0.1,0.0,0.0,0.00000,0.000000,0.000000,0.0


In [12]:
Position_Type.to_pickle('Position_Type.pkl')